In [1]:

from typing import Callable, Tuple, TypeVar, cast, get_origin, get_args, Union, Any, Optional, Dict, List
from components.two_wheels import TwoWheelsV3
from components import (
    CallChannel, 
    ComponentStarter, 
    LoggerComponent,
    BlueToothCarControlSPP_V2,
    start_bluetooth_server_v2b, 
    AngularSpeedControlV3,
    Picamera2V2,
    PitchAngularVelocityController, 
    get_switches,
    MicrophoneComponent
)

from data_collection.data_collection import LoggerSet
from multiprocessing import Manager
import numpy as np
import datetime
import RPi.GPIO as GPIO
GPIO.setmode(GPIO.BOARD)

# from components import numpy_sample_setup
# r, w = numpy_sample_setup(['d', 'd'], [0, 0])

%load_ext autoreload
%autoreload 2

In [3]:
loggerset = LoggerSet('../log/audio_sample'+str(datetime.datetime.now()), overwrite_ok=False)
manager = Manager()

In [4]:

logger_process = ComponentStarter(
    LoggerComponent, 
    manager, 
    init_kwargs=dict(
        loggerset = loggerset
    )
)
two_wheel_process = ComponentStarter(
    TwoWheelsV3, 
    manager, 
    init_kwargs=dict(
        left_pin = 33, 
        right_pin = 32, 
        dir_pins = (16, 18), 
        name='TwoWheelsV3'
    ),
    loop_intervals={'step': 1/100},
    instantiator = TwoWheelsV3.entry

)

angular_speed_control_process = ComponentStarter(
    AngularSpeedControlV3, 
    manager, 
    init_kwargs=dict(
        i2c_address=0x68, 
        bus_num=1,
        name='AngularSpeedControlV3'
    ),
    loop_intervals={'step': 1/100},
    instantiator = AngularSpeedControlV3.entry
)

bluetooth_control_process = ComponentStarter(
    BlueToothCarControlSPP_V2, 
    manager, 
    init_kwargs={},
    loop_intervals={'step': 1/100},
)

bt_ser_out, bt_ser_process_man = start_bluetooth_server_v2b(manager)

mic_com = ComponentStarter(
    MicrophoneComponent, 
    manager, 
    instantiator=MicrophoneComponent.entry, 
    init_kwargs = dict(
        device_indices=[9, 10]
    )
)


In [5]:
two_wheel_process.register_outgoing_rpc(
    dict(log=logger_process.incoming_rpcs['log'])
)

angular_speed_control_process.register_outgoing_rpc(
    dict(log=logger_process.incoming_rpcs['log'])
)

bluetooth_control_process.register_outgoing_rpc(
    dict(log=logger_process.incoming_rpcs['log'])
)
mic_com.register_outgoing_rpc(
    dict(log=logger_process.incoming_rpcs['log'])

)


two_wheel_process.register_incoming_samples(
    angular_speed_control_process.outgoing_sample_readers
)

angular_speed_control_process.register_incoming_samples(
    bluetooth_control_process.outgoing_samples
)

bluetooth_control_process.register_incoming_samples(
    bt_ser_out
)


In [6]:
bt_ser_process_man.start()
bluetooth_control_process.start()
logger_process.start()
two_wheel_process.start()
angular_speed_control_process.start()
mic_com.start()

Making discoverable...
Starting Serial Port Profile...


/home/kawa/projects/car1/src/components/syncronisation/__init__.py:224: UserWarning: the real interval is >= ideal_interval
  warnings.warn("the real interval is >= ideal_interval")
/home/kawa/projects/car1/src/components/syncronisation/__init__.py:224: UserWarning: the real interval is >= ideal_interval
  warnings.warn("the real interval is >= ideal_interval")
/home/kawa/projects/car1/src/components/syncronisation/__init__.py:224: UserWarning: the real interval is >= ideal_interval
  warnings.warn("the real interval is >= ideal_interval")


saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: AngularSpeedControlV3
saving: MicrophoneComponent
NewConnection(/org/bluez/hci0/dev_C8_BD_4D_BA_4F_15, 83)
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: AngularSpeedControlV3
saving: MicrophoneComponent
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: AngularSpeedControlV3
saving: MicrophoneComponent
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: AngularSpeedControlV3
saving: MicrophoneComponent


In [7]:
bt_ser_process_man.terminate()
bluetooth_control_process.process_starter.terminate()
#logger_process.process_starter.terminate()
two_wheel_process.process_starter.terminate()
angular_speed_control_process.process_starter.terminate()
mic_com.process_starter.terminate()



In [8]:
logger_process.incoming_rpcs['log'].call_semaphore

<Semaphore(value=0)>

In [9]:
log = logger_process.incoming_rpcs['get_logger'].call('MicrophoneComponent')()

/home/kawa/projects/car1/src/data_collection/data_collection.py:110: UserWarning: log is not availble after depickling/__setstate__
  warnings.warn('log is not availble after depickling/__setstate__')


/home/kawa/projects/car1/src/data_collection/data_collection.py:110: UserWarning: log is not availble after depickling/__setstate__
  warnings.warn('log is not availble after depickling/__setstate__')


In [10]:
_, l = log
df = l.load_as_df()

# get data

In [11]:
import numpy as np
import datetime
from data_collection.data_collection import Logger
import plotly.express as px
from IPython.display import Audio


In [11]:

# sesspath = '../log/audio_sample2024-08-03 22:09:56.072194'
# l = Logger(sesspath,name='MicrophoneComponent', overwrite_ok=True)
# df = l.load_as_df()

In [12]:
t = df['audio'].apply(lambda x: [np.concatenate(x[i]) for i in x])
t = t.tolist()
c0a = np.concatenate([t_[0] for t_ in t])
c1a = np.concatenate([t_[1] for t_ in t])


In [13]:
Audio(c1a.flatten(), rate=16000)